# 추천 모델이란?

matrix factorization or collaborative filtering이란?\
: item들에 대한 user들의 rating 정보를 바탕으로 유사도를 계산하여 아이템을 추천하는 것.\
(유사 그룹을 찾아서 아이템을 추천: user-based CF / 유사 아이템을 찾아서 추천: item-based CF)

# 아이템 기반 추천시스템 제작

블로그 참조 :

https://kutar37.tistory.com/entry/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%ED%98%91%EC%97%85%ED%95%84%ED%84%B0%EB%A7%81Collaborative-Filtering-%EC%B6%94%EC%B2%9C-%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98-1

https://western-sky.tistory.com/58

https://velog.io/@jumpx2/python-%EC%B5%9C%EA%B7%BC%EC%A0%91-%EC%9D%B4%EC%9B%83-%ED%98%91%EC%97%85-%ED%95%84%ED%84%B0%EB%A7%81

In [ ]:
# 모든 요일 카테고리 분류전
df = pd.read_parquet("C://Users//JEON_SANGEON//codestates//data//cp2_data.parquet")
df

,event_time,month,day,day_name,hour,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 04:00:00+04:00,10,1,Tuesday,4,view,44600062,2103807459595387724,no_cat,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 04:00:00+04:00,10,1,Tuesday,4,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 04:00:01+04:00,10,1,Tuesday,4,view,17200506,2053013559792632471,furniture.living_room.sofa,no_brand,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 04:00:01+04:00,10,1,Tuesday,4,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 04:00:04+04:00,10,1,Tuesday,4,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d
...,...,...,...,...,...,...,...,...,...,...,...,...,...
42448759,2019-11-01 03:59:58+04:00,11,1,Friday,3,view,2300275,2053013560530830019,electronics.camera.video,gopro,527.40,537931532,22c57267-da98-4f28-9a9c-18bb5b385193
42448760,2019-11-01 03:59:58+04:00,11,1,Friday,3,view,10800172,2053013554994348409,appliances.kitchen.multibaker,redmond,61.75,527322328,5054190a-46cb-4211-a8f1-16fc1a060ed8
42448761,2019-11-01 03:59:58+04:00,11,1,Friday,3,view,5701038,2053013553970938175,auto.accessories.player,kenwood,128.70,566280422,05b6c62b-992f-4e8e-91f7-961bcb4719cd
42448762,2019-11-01 03:59:59+04:00,11,1,Friday,3,view,21407424,2053013561579406073,electronics.clocks,tissot,689.85,513118352,4c14bf2a-2820-4504-929d-046356a5a204


In [ ]:
df = df.loc[(df['price']!=0)]

### event_type으로 평점화 진행

In [ ]:
df_v = df.copy()

# 필요한 컬럼만 사용

df_v = df_v[['user_id','category_code','event_type']]
df_v

,user_id,category_code,event_type
0,541312140,no_cat,view
1,554748717,appliances.environment.water_heater,view
2,519107250,furniture.living_room.sofa,view
3,550050854,computers.notebook,view
4,535871217,electronics.smartphone,view
...,...,...,...
42448759,537931532,electronics.camera.video,view
42448760,527322328,appliances.kitchen.multibaker,view
42448761,566280422,auto.accessories.player,view
42448762,513118352,electronics.clocks,view


In [ ]:
df_v = df_v.drop_duplicates()
df_v.event_type.value_counts()

view        6989144
purchase     433130
cart         381975
Name: event_type, dtype: int64

In [ ]:
event_type_strength = {
   'view': 1.5,
   'cart': 1.9, 
   'purchase': 1.6
}

df_v['event_type'] = df_v['event_type'].apply(lambda x: event_type_strength[x])

# 참고
# https://www.kaggle.com/code/gspmoreira/recommender-systems-in-python-101/notebook
# Data munging 부분에서 event_type으로 점수 매김.

In [ ]:
# 선호도 확인을 위한 그룹화 진행

df_v_g = df_v.groupby(['user_id','category_code'])['event_type'].sum().reset_index()
df_v_g

,user_id,category_code,event_type
0,33869381,kids.carriage,1.5
1,183503497,missing,1.5
2,184265397,furniture.living_room.chair,1.5
3,184265397,no_cat,1.5
4,195082191,electronics.audio.headphone,1.5
...,...,...,...
6989477,566280663,electronics.smartphone,1.5
6989478,566280676,furniture.bedroom.bed,1.5
6989479,566280697,electronics.camera.video,1.5
6989480,566280780,furniture.bedroom.set,1.5


In [ ]:
df_v_m = pd.merge(left=df, right=df_v_g,how='left',\
                    on=['user_id','category_code'],sort=False)

df_v_m

,event_time,month,day,day_name,hour,event_type_x,product_id,category_id,category_code,brand,price,user_id,user_session,event_type_y
0,2019-10-01 04:00:00+04:00,10,1,Tuesday,4,view,44600062,2103807459595387724,no_cat,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c,3.1
1,2019-10-01 04:00:00+04:00,10,1,Tuesday,4,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc,1.5
2,2019-10-01 04:00:01+04:00,10,1,Tuesday,4,view,17200506,2053013559792632471,furniture.living_room.sofa,no_brand,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8,1.5
3,2019-10-01 04:00:01+04:00,10,1,Tuesday,4,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713,1.5
4,2019-10-01 04:00:04+04:00,10,1,Tuesday,4,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42380086,2019-11-01 03:59:58+04:00,11,1,Friday,3,view,2300275,2053013560530830019,electronics.camera.video,gopro,527.40,537931532,22c57267-da98-4f28-9a9c-18bb5b385193,1.5
42380087,2019-11-01 03:59:58+04:00,11,1,Friday,3,view,10800172,2053013554994348409,appliances.kitchen.multibaker,redmond,61.75,527322328,5054190a-46cb-4211-a8f1-16fc1a060ed8,1.5
42380088,2019-11-01 03:59:58+04:00,11,1,Friday,3,view,5701038,2053013553970938175,auto.accessories.player,kenwood,128.70,566280422,05b6c62b-992f-4e8e-91f7-961bcb4719cd,1.5
42380089,2019-11-01 03:59:59+04:00,11,1,Friday,3,view,21407424,2053013561579406073,electronics.clocks,tissot,689.85,513118352,4c14bf2a-2820-4504-929d-046356a5a204,1.5


In [ ]:
df_day = df_v_m.loc[(df_v_m['day_name']=='Friday')|
                (df_v_m['day_name']=='Saturday')|
                (df_v_m['day_name']=='Sunday')
                ]

In [ ]:
df_day['category_code'] = df_day["category_code"].str.split(".").str[1]

In [ ]:
df_cat = df_day.loc[(df_day['category_code']=='smartphone')|
                    (df_day['category_code']=='video')|
                    (df_day['category_code']=='kitchen')|
                    (df_day['category_code']=='notebook')
                    ]

In [ ]:
df_cat=df_cat.reset_index(drop=True)
df_cat

,event_time,month,day,day_name,hour,event_type_x,product_id,category_id,category_code,brand,price,user_id,user_session,event_type_y
0,2019-10-04 00:00:00+04:00,10,4,Friday,0,purchase,1002544,2053013555631882655,smartphone,apple,462.39,540570839,a998827d-5c49-4bb5-a175-050c381f5ca5,5.0
1,2019-10-04 00:00:00+04:00,10,4,Friday,0,view,1004858,2053013555631882655,smartphone,samsung,132.02,548264984,6e0d1c2b-254e-4fb8-b95d-f24e04619fc4,1.5
2,2019-10-04 00:00:00+04:00,10,4,Friday,0,view,1004433,2053013555631882655,smartphone,samsung,254.57,556511103,02226436-9a1a-4b10-8b37-748ec545aca0,1.5
3,2019-10-04 00:00:00+04:00,10,4,Friday,0,view,1002544,2053013555631882655,smartphone,apple,462.39,538176090,e9837c18-b9b9-471b-98ad-539833e309e3,3.1
4,2019-10-04 00:00:00+04:00,10,4,Friday,0,view,31501046,2053013558031024687,kitchen,luminarc,108.09,551285153,2d1a08e4-d227-4ea9-89bd-8e0ef3e6b5c8,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7658866,2019-11-01 03:59:56+04:00,11,1,Friday,3,view,1005008,2053013555631882655,smartphone,xiaomi,94.46,557202408,2169c430-9acc-4db8-8061-ac70ff0c66ea,1.5
7658867,2019-11-01 03:59:56+04:00,11,1,Friday,3,view,1003936,2053013555631882655,smartphone,xiaomi,437.33,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33,1.5
7658868,2019-11-01 03:59:58+04:00,11,1,Friday,3,view,1004870,2053013555631882655,smartphone,samsung,275.25,518956209,6764041a-9285-4869-8a32-a79adf31d212,1.5
7658869,2019-11-01 03:59:58+04:00,11,1,Friday,3,view,2702331,2053013563911439225,kitchen,lg,527.43,524356542,153f9818-4d32-4e8b-ba9f-f355094e8ae4,1.5


In [ ]:
# 평점화 진행 후 데이터 parquet화

df_cat.to_parquet('last.parquet', engine="pyarrow", compression="gzip")

# 모델링

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import matplotlib as mpl
from datetime import datetime
from datetime import timedelta

# Windows
mpl.rc("font", family='Malgun Gothic')

warnings.filterwarnings("ignore")
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [2]:
df = pd.read_parquet("C://Users//JEON_SANGEON//codestates//data//last.parquet")
df

,event_time,month,day,day_name,hour,event_type_x,product_id,category_id,category_code,brand,price,user_id,user_session,event_type_y
0,2019-10-04 00:00:00+04:00,10,4,Friday,0,purchase,1002544,2053013555631882655,smartphone,apple,462.39,540570839,a998827d-5c49-4bb5-a175-050c381f5ca5,5.0
1,2019-10-04 00:00:00+04:00,10,4,Friday,0,view,1004858,2053013555631882655,smartphone,samsung,132.02,548264984,6e0d1c2b-254e-4fb8-b95d-f24e04619fc4,1.5
2,2019-10-04 00:00:00+04:00,10,4,Friday,0,view,1004433,2053013555631882655,smartphone,samsung,254.57,556511103,02226436-9a1a-4b10-8b37-748ec545aca0,1.5
3,2019-10-04 00:00:00+04:00,10,4,Friday,0,view,1002544,2053013555631882655,smartphone,apple,462.39,538176090,e9837c18-b9b9-471b-98ad-539833e309e3,3.1
4,2019-10-04 00:00:00+04:00,10,4,Friday,0,view,31501046,2053013558031024687,kitchen,luminarc,108.09,551285153,2d1a08e4-d227-4ea9-89bd-8e0ef3e6b5c8,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7658866,2019-11-01 03:59:56+04:00,11,1,Friday,3,view,1005008,2053013555631882655,smartphone,xiaomi,94.46,557202408,2169c430-9acc-4db8-8061-ac70ff0c66ea,1.5
7658867,2019-11-01 03:59:56+04:00,11,1,Friday,3,view,1003936,2053013555631882655,smartphone,xiaomi,437.33,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33,1.5
7658868,2019-11-01 03:59:58+04:00,11,1,Friday,3,view,1004870,2053013555631882655,smartphone,samsung,275.25,518956209,6764041a-9285-4869-8a32-a79adf31d212,1.5
7658869,2019-11-01 03:59:58+04:00,11,1,Friday,3,view,2702331,2053013563911439225,kitchen,lg,527.43,524356542,153f9818-4d32-4e8b-ba9f-f355094e8ae4,1.5


In [3]:
df = df.rename(columns={'event_type_y': 'rating'})

참조 : 
https://github.com/journey101/Recommendation_CF_Hybrid_Modeling_with_Python_Surprise/blob/main/Recommendation_CF_Hybrid_modeling_for_Fashion_update(210621).ipynb

In [4]:
# rating을 value로 놓고 진행함.

# product_id 관심도를 활용하여 value_average 추가 후 merge

mean_df = df[['product_id','rating']].groupby(by='product_id').mean()
mean_df = mean_df.reset_index()
mean_df = mean_df.rename(columns={'rating': 'rating_average'})
mean_df

,product_id,rating_average
0,1000978,2.344559
1,1001588,2.405342
2,1001606,1.500000
3,1002042,3.005634
4,1002062,2.521239
...,...,...
15996,46800030,1.605648
15997,46800031,1.667697
15998,46800034,1.500000
15999,46800035,1.500000


In [18]:
mean_df['rating_average'].value_counts()

1.500000    3686
2.300000     108
2.033333     107
3.100000     107
5.000000      74
            ... 
1.865854       1
2.106667       1
1.800840       1
2.287652       1
1.667697       1
Name: rating_average, Length: 8187, dtype: int64

In [5]:
df_r_m = pd.merge(df, mean_df, how='left', on='product_id')
df_r_m['user_id'] = df_r_m['user_id'].astype(int)
df_r_m['product_id'] = df_r_m['product_id'].astype(int)
# df_r_merge.dtypes

df_r_m.head()

,event_time,month,day,day_name,hour,event_type_x,product_id,category_id,category_code,brand,price,user_id,user_session,rating,rating_average
0,2019-10-04 00:00:00+04:00,10,4,Friday,0,purchase,1002544,2053013555631882655,smartphone,apple,462.39,540570839,a998827d-5c49-4bb5-a175-050c381f5ca5,5.0,2.960048
1,2019-10-04 00:00:00+04:00,10,4,Friday,0,view,1004858,2053013555631882655,smartphone,samsung,132.02,548264984,6e0d1c2b-254e-4fb8-b95d-f24e04619fc4,1.5,3.108415
2,2019-10-04 00:00:00+04:00,10,4,Friday,0,view,1004433,2053013555631882655,smartphone,samsung,254.57,556511103,02226436-9a1a-4b10-8b37-748ec545aca0,1.5,2.373963
3,2019-10-04 00:00:00+04:00,10,4,Friday,0,view,1002544,2053013555631882655,smartphone,apple,462.39,538176090,e9837c18-b9b9-471b-98ad-539833e309e3,3.1,2.960048
4,2019-10-04 00:00:00+04:00,10,4,Friday,0,view,31501046,2053013558031024687,kitchen,luminarc,108.09,551285153,2d1a08e4-d227-4ea9-89bd-8e0ef3e6b5c8,1.5,1.655698


In [11]:
df_r_m.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7658871 entries, 0 to 7658870
Data columns (total 15 columns):
 #   Column          Dtype                     
---  ------          -----                     
 0   event_time      datetime64[ns, Asia/Dubai]
 1   month           int64                     
 2   day             int64                     
 3   day_name        object                    
 4   hour            int64                     
 5   event_type_x    object                    
 6   product_id      int32                     
 7   category_id     int64                     
 8   category_code   object                    
 9   brand           object                    
 10  price           float64                   
 11  user_id         int32                     
 12  user_session    object                    
 13  rating          float64                   
 14  rating_average  float64                   
dtypes: datetime64[ns, Asia/Dubai](1), float64(3), int32(2), int64(4), 

In [6]:
df_r_m['rating'].value_counts()

1.5    4688667
5.0    1620976
3.4     896815
3.1     452259
1.9         86
1.6         35
3.5         33
Name: rating, dtype: int64

# 베이스라인 모형

이 과정은 evaluate 명령으로 단축할 수도 있다.

surprise 패키지는 베이스라인 모형을 위한 BaselineOnly 클래스를 제공한다.

In [12]:
from surprise import SVD, BaselineOnly, SVDpp, Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import KNNBaseline
from surprise.model_selection import GridSearchCV
from surprise import KNNBasic
from surprise import accuracy
from surprise import dump
from surprise.model_selection import KFold
from surprise import NormalPredictor
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from surprise.model_selection import train_test_split

# 모델 분석방법

# Model-based Collaborative Filtering

- Clustering-based Algorithm
- Matrix Factorization Algorithm
- Deep Learning

세가지로 분류되는데 그 중 Clustering-based Algorithm에 surprise KNN을 활용예정


surprise 모듈 임포트

Surprise
- 사용자 아이디, 아이템 아이디, 평점 데이터가 로우 레벨로 된 데이터 세트에 적용이 가능한 모듈

train, test set

In [13]:
# surprise 모듈의 훈련,테스트 모듈 나누는것과 비슷한 방법으로 데이터 분리 진행

# train, test split

# 데이터 랜덤하게 섞기
df_r_m = df.sample(frac=1)

# 7:3 비율로 나누기
cutoff = int(.7 * len(df_r_m))
train_r = df_r_m[:cutoff]
test_r = df_r_m[cutoff:]


train_r = train_r[['user_id', 'product_id', 'rating', 'category_code', 'brand']]
test_r = test_r[['user_id', 'product_id', 'rating', 'category_code', 'brand']]
print("train_size:", len(train_r), "test_size:", len(test_r))

train_r.head()

train_size: 5361209 test_size: 2297662


,user_id,product_id,rating,category_code,brand
6471075,534275871,1004708,3.1,smartphone,huawei
1560517,544436647,1004659,1.5,smartphone,samsung
2827098,535886556,1004250,5.0,smartphone,apple
3554791,522235312,2800636,1.5,kitchen,indesit
608023,525624569,3601255,1.5,kitchen,no_brand


In [14]:
def convert_traintest_dataframe_forsurprise(training_dataframe, testing_dataframe):
    reader = Reader(rating_scale=(1, 5))
    trainset = Dataset.load_from_df(training_dataframe[['user_id', 'product_id', 'rating']], reader)
    testset = Dataset.load_from_df(testing_dataframe[['user_id', 'product_id', 'rating']], reader)
    
    trainset = trainset.construct_trainset(trainset.raw_ratings)
    testset = testset.construct_testset(testset.raw_ratings)
    
    return trainset, testset

trainset, testset = convert_traintest_dataframe_forsurprise(train_r, test_r)

Surprise 추천알고리즘 클래스

KNNBaseline - 기준 등급 을 고려한 기본 협업 필터링 알고리즘 입니다.

SVD - Netflix Prize 기간 동안 Simon Funk 가 대중화한 유명한 SVD 알고리즘입니다. 기준선을 사용하지 않는 경우 이는 확률적 행렬 분해[salakhutdinov2008a]와 동일합니다(아래 참고 참조).

SVDpp - SVD++ 알고리즘은 SVD 암시적 등급 고려의 확장 입니다.


https://surprise.readthedocs.io/en/stable/prediction_algorithms_package.html

참고 : 
https://github.com/prakruti-joshi/Movie-Recommendation-System/blob/master/Code/cold_start_analysis.ipynb

## (1) Knn-baseline 모델

- 기준 등급 을 고려한 기본 협업 필터링 알고리즘 입니다.

surprise.prediction_algorithms.knns.KNNBaseline( k=40 , min_k=1 , sim_options={} , bsl_options={} , verbose=True , **kwargs )

product_id 기반 모델

In [15]:
# basic collaborative filtering algorithm taking into account a baseline rating.
# content-based filtering (CBF)
# 참조: https://surprise.readthedocs.io/en/stable/knn_inspired.html


sim_options = {'name': 'cosine',
               'user_based': False  # content(item)-based similarity
               }
knnbaseline_algo_i = KNNBaseline(sim_options=sim_options)

knnbaseline_algo_i.fit(trainset)
knnbaseline_predictions_i = knnbaseline_algo_i.test(testset)

file_name = 'KnnBaseline_model_i'
dump.dump(file_name, algo=knnbaseline_predictions_i)
# _, loaded_algo = dump.load(file_name)

accuracy.rmse(knnbaseline_predictions_i)
accuracy.mae(knnbaseline_predictions_i)
print("Done!")

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.4316
MAE:  0.2192
Done!


user_id 기반 모델 (메모리 초과로 진행불가)

In [ ]:
# basic collaborative filtering algorithm taking into account a baseline rating.
# user-based filtering (CBF)
# 참조: https://surprise.readthedocs.io/en/stable/knn_inspired.html

# sim_options = {'name': 'cosine',
#                'user_based': True  # user-based similarity
#                }
# knnbaseline_algo_u = KNNBaseline(sim_options=sim_options)

# knnbaseline_algo_u.fit(trainset)
# knnbaseline_predictions_u = knnbaseline_algo_u.test(testset)

# file_name = 'KnnBaseline_model_u'
# dump.dump(file_name, algo=knnbaseline_predictions_u)
# # _, loaded_algo = dump.load(file_name)

# accuracy.rmse(knnbaseline_predictions_u)
# accuracy.mae(knnbaseline_predictions_u)
# print("Done!")

In [16]:
knnbaseline_predictions_i = knnbaseline_algo_i.test(testset)
knnbaseline_predictions_i[:3]

[Prediction(uid=564085761, iid=1004246, r_ui=1.5, est=1.7705682770890259, details={'actual_k': 6, 'was_impossible': False}),
 Prediction(uid=558992205, iid=1003317, r_ui=1.5, est=2.3920117556500182, details={'was_impossible': False}),
 Prediction(uid=552065728, iid=3600145, r_ui=1.5, est=1.5, details={'actual_k': 1, 'was_impossible': False})]

In [17]:
# uid, iid, r_ui(실제 평점) ,est(예측평점) 추출

[(pred.uid, pred.iid, pred.est) for pred in knnbaseline_predictions_i[:3]]

[(564085761, 1004246, 1.7705682770890259),
 (558992205, 1003317, 2.3920117556500182),
 (552065728, 3600145, 1.5)]

In [18]:
# uid = 564085761 에 대한 유사도 높은 예측 iid, est 조회. 
# 참고: https://hmkim312.github.io/posts/%EC%84%9C%ED%94%84%EB%9D%BC%EC%9D%B4%EC%A6%88(Surprise)%EB%A5%BC_%EC%82%AC%EC%9A%A9%ED%95%9C_%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C(Recommendations)/

pred_df = pd.DataFrame(knnbaseline_predictions_i)
pred_df[(pred_df['uid']==564085761)]

,uid,iid,r_ui,est,details
0,564085761,1004246,1.5,1.770568,"{'actual_k': 6, 'was_impossible': False}"
819243,564085761,1004258,1.5,1.908919,"{'actual_k': 9, 'was_impossible': False}"
1553182,564085761,1003316,1.5,1.840554,"{'actual_k': 9, 'was_impossible': False}"
1952502,564085761,1004252,1.5,1.566236,"{'actual_k': 6, 'was_impossible': False}"
2187687,564085761,31501004,1.5,1.254045,"{'actual_k': 4, 'was_impossible': False}"


## (2) svd, svdpp (CF) 모델

In [19]:
 ## svd 
 ## 참고: https://surprise.readthedocs.io/en/stable/matrix_factorization.html#matrix-factorization-based-algorithms

svd_algo = SVD()

svd_algo.fit(trainset)
svd_predictions = svd_algo.test(testset)

file_name = 'svd_model'
dump.dump(file_name, algo=svd_algo)
# _, loaded_algo = dump.load(file_name)

accuracy.rmse(svd_predictions)
accuracy.mae(svd_predictions)
print("Done!")

RMSE: 0.4463
MAE:  0.3396
Done!


In [20]:
# matrix factorization/ collaborative filtering(CF)
 ## svd++ model : taking into account implicit ratings 
 ## 참고: https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVDpp

svdpp_algo = SVDpp()

svdpp_algo.fit(trainset)
svdpp_predictions = svdpp_algo.test(testset)

file_name = 'svdpp_model'
dump.dump(file_name, algo=svdpp_algo)
# _, loaded_algo = dump.load(file_name)

accuracy.rmse(svdpp_predictions)
accuracy.mae(svdpp_predictions)
print("Done!")

RMSE: 2.8310
MAE:  2.4428
Done!


## (3) gridsearchCV 모델 오래걸려서 중단

In [ ]:
# from surprise import SVD
# from surprise import Dataset
# from surprise import Reader
# from surprise.model_selection import GridSearchCV

# param_grid = {'n_epochs': [5, 20], 'lr_all': [0.002, 0.005],
#               'reg_all': [0.4, 0.6]}
#               # lr_all – The learning rate for all parameters. Default is 0.005.
#               # reg_all – The regularization term for all parameters. Default is 0.02.
              
# gs = GridSearchCV(SVDpp, param_grid, measures=['rmse', 'mae'], cv=5) 

# reader = Reader()

# # The columns must correspond to user id, product_id and ratings (in that order).
# train_data = Dataset.load_from_df(train_r[['user_id', 'product_id', 'rating']], reader)

# gs.fit(train_data)

# # best RMSE score
# print("gs.best_score(rmse): ", gs.best_score['rmse'])

# # combination of parameters that gave the best RMSE score
# print("gs.best_params: ", gs.best_params['rmse'])

In [ ]:
# from surprise import accuracy

# # best model fit 
# # We can now use the algorithm that yields the best rmse:
# GS_SVDpp_best = gs.best_estimator['rmse']

# # retrain on the whole set train
# trainset = train_data.build_full_trainset()
# GS_SVDpp_best.fit(trainset)

# # Compute biased accuracy on train (validation set)
# predictions = GS_SVDpp_best.test(trainset.build_testset())
# print('validation test,', end='\n')
# accuracy.rmse(predictions)
# accuracy.mae(predictions)
# # 참고:https://surprise.readthedocs.io/en/stable/FAQ.html#how-to-save-some-data-for-unbiased-accuracy-estimation
# ## 참고: https://surprise.readthedocs.io/en/stable/getting_started.html#train-on-whole-trainset

In [ ]:
# # Compute biased accuracy on train (validation set)
# predictions = GS_SVDpp_best.test(testset)
# print('testset accuracy,', end='\n')
# accuracy.rmse(predictions)
# accuracy.mae(predictions)

### 1-2) evaluation (knn/svd/svdpp 비교)

- knnbaseline (item_based)
- SVD (CF)
- SVDpp (CF)

prediction, rmse, mae, precision, recall, f-measure, ndcg_score 계산값

In [22]:
# predictions 
knn_i_pred = knnbaseline_algo_i.test(testset)
svd_pred = svd_algo.test(testset)
svdpp_pred = svdpp_algo.test(testset)

In [23]:
# test_rmse, test_mae
knn_i_rmse = accuracy.rmse(knn_i_pred)
knn_i_mae = accuracy.mae(knn_i_pred)
svd_rmse = accuracy.rmse(svd_pred)
svd_mae = accuracy.mae(svd_pred)
svdpp_rmse = accuracy.rmse(svdpp_pred)
svdpp_mae = accuracy.mae(svdpp_pred)

RMSE: 0.4316
MAE:  0.2192
RMSE: 0.4463
MAE:  0.3396
RMSE: 2.8310
MAE:  2.4428


In [24]:
# recall & precision @ k function
# 참고: https://surprise.readthedocs.io/en/stable/FAQ.html?highlight=recall#how-to-compute-precision-k-and-recall-k

from collections import defaultdict
from surprise import Dataset
from surprise import SVD
from surprise.model_selection import KFold


def precision_recall_at_k(predictions, k, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

In [25]:
# knn_i 모델 precision&recall
knn_i_prec, knn_i_re = precision_recall_at_k(knn_i_pred, k=5, threshold=4)

# Precision and recall can then be averaged over all users
knn_i_prec_avg = sum(prec for prec in knn_i_prec.values()) / len(knn_i_prec)
print(knn_i_prec_avg)
knn_i_re_avg = sum(rec for rec in knn_i_re.values()) / len(knn_i_re)
print(knn_i_re_avg)

0.12248232875149963
0.10366478709755178


In [26]:

# svd precision&recall
svd_prec, svd_re = precision_recall_at_k(svd_pred, k=5, threshold=4)

# Precision and recall can then be averaged over all users
svd_prec_avg = sum(prec for prec in svd_prec.values()) / len(svd_prec)
print(svd_prec_avg)
svd_re_avg = sum(rec for rec in svd_re.values()) / len(svd_re)
print(svd_re_avg)

0.11253895689540903
0.08951768021956362


In [27]:
# svdpp precision&recall
svdpp_prec, svdpp_re = precision_recall_at_k(svdpp_pred, k=5, threshold=4)

# Precision and recall can then be averaged over all users
svdpp_prec_avg = sum(prec for prec in svdpp_prec.values()) / len(svdpp_prec)
print(svdpp_prec_avg)
svdpp_re_avg = sum(rec for rec in svdpp_re.values()) / len(svdpp_re)
print(svdpp_re_avg)

0.12492971574953085
0.10966208195674963


In [28]:
# model별 test prediction  데이터 
# row: Knnbaseline(item based), Knnbaseline(user based), SVD(CF), SVDpp(CF), gridsearch(SVDpp)
# columns: test RMSE, test MAE, test_time, Precision, recall
# hybrid 모델 수치는 SVDpp(CF)모델과 Knnbaseline(item-based)의 평균값. 

test_prediction = pd.DataFrame({"Algorithm": ["KnnBaseline(item-based)" ,  
                                              "SVD(CF)",
                                              "SVDpp(CF)"]
                                ,"test_rmse": [knn_i_rmse , svd_rmse , svdpp_rmse]
                                ,"test_mae":[knn_i_mae , svd_mae, svdpp_mae]
                                ,"Precision":[knn_i_prec_avg ,svd_prec_avg ,svdpp_prec_avg]
                                ,"recall":[knn_i_re_avg ,svd_re_avg ,svdpp_re_avg]
                                })

test_prediction

,Algorithm,test_rmse,test_mae,Precision,recall
0,KnnBaseline(item-based),0.431577,0.219185,0.122482,0.103665
1,SVD(CF),0.446251,0.339567,0.112539,0.089518
2,SVDpp(CF),2.830991,2.442792,0.124930,0.109662
